In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression

In [100]:
user = pd.read_csv('user_final.csv', sep='\t')

In [101]:
user.head(10)

,Unnamed: 0,user_id,age_range,gender,click,add,buy,save,userTotalAction,click_ratio_,...,index,unique_item,unique_cat,unique_seller,unique_brand,unique_active_day,day_count,month_count,day_norm,rep_buyer
0,0,376517,6.0,1.0,73.0,0.0,5.0,1.0,79.0,0.924051,...,376517,39,17,20,20,10,10,3.333,0.592154,1.0
1,1,234512,5.0,0.0,47.0,0.0,7.0,2.0,56.0,0.839286,...,234512,29,18,21,20,11,11,3.667,0.651370,1.0
2,2,344532,5.0,0.0,64.0,0.0,8.0,19.0,91.0,0.703297,...,344532,54,25,20,21,14,14,2.800,0.829016,1.0
3,3,186135,5.0,0.0,193.0,0.0,2.0,5.0,200.0,0.965000,...,186135,21,11,11,10,5,5,5.000,0.296077,0.0
4,4,30230,5.0,0.0,29.0,0.0,3.0,0.0,32.0,0.906250,...,30230,24,13,14,14,13,13,2.167,0.769801,0.0
5,5,272389,6.0,1.0,25.0,0.0,1.0,0.0,26.0,0.961538,...,272389,23,9,5,5,2,2,2.000,0.118431,0.0
6,6,281071,4.0,0.0,606.0,0.0,24.0,13.0,643.0,0.942457,...,281071,420,94,106,111,47,47,6.714,2.783125,1.0
7,7,139859,7.0,0.0,42.0,0.0,1.0,0.0,43.0,0.976744,...,139859,30,22,24,23,17,17,2.429,1.006662,0.0
8,8,198411,5.0,1.0,80.0,0.0,10.0,9.0,99.0,0.808081,...,198411,61,32,36,38,23,23,3.286,1.361955,1.0
9,9,67037,4.0,1.0,70.0,0.0,11.0,10.0,91.0,0.769231,...,67037,69,38,35,32,18,18,2.571,1.065878,1.0


In [102]:
seller = pd.read_csv('seller_final.csv', sep='\t')
seller.head()

,Unnamed: 0,seller_id,click,cart,buy,fav,sellerTotalAction,click_to_buy_ratio,cart_to_buy_ratio,fav_to_buy_ratio,...,item_count,cat_count,brand_count,click_ratio,cart_ratio,buy_ratio,fav_ratio,item_ratio,cat_ratio,brand_ratio
0,0,1,308236.0,444.0,17705.0,12755.0,339140.0,-2.856965,3.683585,0.327902,...,2977,44,2,0.069011,0.008536,0.018073,0.021270,0.002730,0.026538,0.000237
1,1,2,2030.0,8.0,189.0,144.0,2371.0,-2.369259,3.049799,0.270290,...,154,10,1,0.002128,0.000193,0.000380,0.000544,0.000141,0.006031,0.000118
2,2,3,2399.0,4.0,67.0,175.0,2645.0,-3.563716,2.610070,-0.950976,...,171,4,1,0.002602,0.000129,0.000153,0.000643,0.000157,0.002413,0.000118
3,3,4,2646.0,2.0,294.0,164.0,3106.0,-2.194207,4.588363,0.581030,...,155,7,2,0.003265,0.000064,0.000474,0.000655,0.000142,0.004222,0.000237
4,4,5,7483.0,9.0,144.0,556.0,8192.0,-3.943789,2.674149,-1.345831,...,660,23,1,0.008340,0.000290,0.000283,0.001962,0.000605,0.013872,0.000118


In [103]:
train = pd.read_csv('train_format1.csv')

In [104]:
train.head(10)

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0
5,362112,2618,0
6,34944,2051,0
7,231552,3828,1
8,231552,2124,0
9,232320,1168,0


In [140]:
train.label.value_counts()

0    244912
1     15952
Name: label, dtype: int64

In [105]:
test = pd.read_csv('test_format1.csv')
test.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [106]:
train.rename(columns={'merchant_id':'seller_id'},inplace=True)

In [107]:
train.head()

,user_id,seller_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [108]:
train_set = pd.merge(train, user, how='left', on=['user_id'])

In [109]:
train_set.head()

,user_id,seller_id,label,Unnamed: 0,age_range,gender,click,add,buy,save,...,index,unique_item,unique_cat,unique_seller,unique_brand,unique_active_day,day_count,month_count,day_norm,rep_buyer
0,34176,3906,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,34176.0,256.0,45.0,109.0,108.0,47.0,47.0,6.714,2.783125,1.0
1,34176,121,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,34176.0,256.0,45.0,109.0,108.0,47.0,47.0,6.714,2.783125,1.0
2,34176,4356,1,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,34176.0,256.0,45.0,109.0,108.0,47.0,47.0,6.714,2.783125,1.0
3,34176,2217,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,34176.0,256.0,45.0,109.0,108.0,47.0,47.0,6.714,2.783125,1.0
4,230784,4818,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
train_set = pd.merge(train_set, seller, how='left', on=['seller_id'])

In [111]:
train_set.head()

,user_id,seller_id,label,Unnamed: 0_x,age_range,gender,click_x,add,buy_x,save,...,item_count,cat_count,brand_count,click_ratio,cart_ratio,buy_ratio,fav_ratio,item_ratio,cat_ratio,brand_ratio
0,34176,3906,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,308,20,1,0.013311,0.000805,0.000731,0.003046,0.000282,0.012063,0.000118
1,34176,121,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,1179,26,1,0.024546,0.002706,0.006493,0.005925,0.001081,0.015682,0.000118
2,34176,4356,1,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,67,15,1,0.005115,0.000387,0.001591,0.000771,0.000061,0.009047,0.000118
3,34176,2217,0,173604.0,6.0,0.0,410.0,0.0,34.0,7.0,...,377,5,1,0.036852,0.002738,0.007811,0.013006,0.000346,0.003016,0.000118
4,230784,4818,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,461,27,1,0.017001,0.002835,0.003704,0.004198,0.000423,0.016285,0.000118


In [112]:
test.rename(columns={'merchant_id':'seller_id'},inplace=True)

In [113]:
test.head(10)

,user_id,seller_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN
5,33408,98,NaN
6,230016,1742,NaN
7,164736,598,NaN
8,164736,1963,NaN
9,164736,2634,NaN


In [114]:
test_set = pd.merge(test, user, how='left', on=['user_id'])

In [115]:
test_set = pd.merge(test_set, seller, how='left', on=['seller_id'])

In [116]:
test_set.head()

,user_id,seller_id,prob,Unnamed: 0_x,age_range,gender,click_x,add,buy_x,save,...,item_count,cat_count,brand_count,click_ratio,cart_ratio,buy_ratio,fav_ratio,item_ratio,cat_ratio,brand_ratio
0,163968,4605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,223,31,1,0.034002,0.001611,0.010508,0.010817,0.000205,0.018697,0.000118
1,360576,1581,NaN,68879.0,2.0,2.0,71.0,0.0,6.0,0.0,...,57,8,1,0.001741,0.000097,0.000752,0.000673,0.000052,0.004825,0.000118
2,98688,1964,NaN,184138.0,6.0,0.0,51.0,0.0,5.0,0.0,...,113,22,1,0.010751,0.000870,0.003270,0.002737,0.000104,0.013269,0.000118
3,98688,3645,NaN,184138.0,6.0,0.0,51.0,0.0,5.0,0.0,...,440,21,1,0.020061,0.000902,0.002065,0.004682,0.000404,0.012666,0.000118
4,295296,3361,NaN,184077.0,2.0,1.0,162.0,0.0,7.0,7.0,...,89,13,2,0.005011,0.000354,0.000457,0.000793,0.000082,0.007841,0.000237


In [117]:
train_set.isna().any()

user_id                 False
seller_id               False
label                   False
Unnamed: 0_x             True
age_range                True
gender                   True
click_x                  True
add                      True
buy_x                    True
save                     True
userTotalAction          True
click_ratio_             True
add_ratio_               True
buy_ratio_               True
save_ratio_              True
click_conversion         True
clickconversion_diff     True
add_conversion           True
addconversion_diff       True
save_conversion          True
saveconversion_diff      True
index                    True
unique_item              True
unique_cat               True
unique_seller            True
unique_brand             True
unique_active_day        True
day_count                True
month_count              True
day_norm                 True
rep_buyer                True
Unnamed: 0_y            False
click_y                 False
cart      

In [118]:
# train_set = train_set.fillna(-1)
train_set = train_set.dropna()

In [119]:
train_set.isna().any()

user_id                 False
seller_id               False
label                   False
Unnamed: 0_x            False
age_range               False
gender                  False
click_x                 False
add                     False
buy_x                   False
save                    False
userTotalAction         False
click_ratio_            False
add_ratio_              False
buy_ratio_              False
save_ratio_             False
click_conversion        False
clickconversion_diff    False
add_conversion          False
addconversion_diff      False
save_conversion         False
saveconversion_diff     False
index                   False
unique_item             False
unique_cat              False
unique_seller           False
unique_brand            False
unique_active_day       False
day_count               False
month_count             False
day_norm                False
rep_buyer               False
Unnamed: 0_y            False
click_y                 False
cart      

In [120]:
x = train_set.loc[:,train_set.columns != 'label']
y = train_set.loc[:,train_set.columns == 'label']
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1992)

In [ ]:
#params=[
    #{'penalty':['l1'],
#'C':[100,1000],
#'solver':['liblinear']},
    #{'penalty':['l2'],
#'C':[100,1000],
#'solver':['lbfgs']}]
#clf = LogisticRegression(random_state=2018, max_iter=1000,  verbose=2)
#grid = GridSearchCV(clf, params, scoring='roc_auc',cv=10, verbose=2)
#grid.fit(X_train, y_train) 
#lr=grid.best_estimator_

In [122]:
params=[
    {'penalty':['l1'],
    'C':[100,1000],
    'solver':['liblinear']}]
clf = LogisticRegression(random_state=1992, max_iter=1000,  verbose=2)
grid = GridSearchCV(clf, params, scoring='roc_auc',cv=10, verbose=2)
grid.fit(X_train, y_train) 
lr=grid.best_estimator_

Fitting 10 folds for each of 2 candidates, totalling 20 fits
[CV] C=100, solver=liblinear, penalty=l1 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/nfshome/jl9760/.conda/envs/myEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibLinear][CV] .............. C=100, solver=liblinear, penalty=l1, total=  29.4s
[CV] C=100, solver=liblinear, penalty=l1 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.9s remaining:    0.0s
/nfshome/jl9760/.conda/envs/myEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibLinear][CV] .............. C=100, solver=liblinear, penalty=l1, total=  31.7s
[CV] C=100, solver=liblinear, penalty=l1 .............................


...

[LibLinear][CV] ............. C=1000, solver=liblinear, penalty=l1, total=  41.3s
[CV] C=1000, solver=liblinear, penalty=l1 ............................


/nfshome/jl9760/.conda/envs/myEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibLinear][CV] ............. C=1000, solver=liblinear, penalty=l1, total=  44.5s


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 12.7min finished
/nfshome/jl9760/.conda/envs/myEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibLinear]

In [76]:
lr.fit(X_train, y_train)

/nfshome/jl9760/.conda/envs/myEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibLinear]

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=1992, solver='liblinear',
          tol=0.0001, verbose=2, warm_start=False)

In [131]:
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.94


In [129]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[38551     0]
 [ 2460     0]]


In [133]:
print(grid.best_score_) 
print(grid.best_params_)

print(grid.best_estimator_) 

0.6040970572948003
{'C': 100, 'solver': 'liblinear', 'penalty': 'l1'}
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=1992, solver='liblinear',
          tol=0.0001, verbose=2, warm_start=False)


In [135]:
y_train[y_train.label == 0].count()

label    153842
dtype: int64

In [136]:
y_train[y_train.label == 1].count()

label    10202
dtype: int64

In [137]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(164044, 53) (164044, 1) (41011, 53) (41011, 1)
